<a href="https://colab.research.google.com/github/mggg/Training_Materials_25/blob/main/notebooks/technical/Thursday/Tutorial_2_Running_elections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Elections

Elections are the systems or algorithms by which a `PreferenceProfile`, or collection of ballots, is converted into an outcome. There are infinitely many different possible election methods, whether the output is a single winner, a set of winners, or a consensus ranking. VoteKit has a host of built-in election methods, as well as the functionality to let you create your own system of election. By the end of this section, you will have been introduced to the STV and Borda elections, learned about the `Election` object, and created your own election type.

In [ ]:
import pandas as pd

# To load the Minneapolis CVR data
from votekit.cvr_loaders import load_csv
from votekit.cleaning import remove_and_condense

from votekit.elections import STV

# Used in this notebook to make some synthetic Preference Profiles
from votekit.ballot import Ballot
from votekit.pref_profile import PreferenceProfile

# Used to modify STV transfer method
from votekit.elections import random_transfer


# Used for exploring Borda elections
from votekit.elections import Borda
import votekit.ballot_generator as bg


# Used for making custom elections
from votekit.elections import RankingElection, ElectionState
from votekit.cleaning import remove_cand
import random
from fractions import Fraction


## STV (Single Transferable Vote)

We are going to start with running through the Minneapolis 2013 mayoral election.
This was an election where there were 35 candidates running for one seat in an IRV election
which is equivalent to a single-winner STV election.

We have a copy of the **cast vote record (CVR)** in our tutorial data for the VoteKit package,
so we will load that here.

In [2]:
minneapolis_profile = load_csv("https://raw.githubusercontent.com/mggg/VoteKit/refs/heads/main/examples/data/mn_2013_cast_vote_record.csv")
minneapolis_profile = remove_and_condense(["undervote", "overvote", "UWI"], minneapolis_profile)
minneapolis_profile

Profile has been cleaned
Profile contains rankings: True
Maximum ranking length: 3
Profile contains scores: False
Candidates: ('JACKIE CHERRYHOMES', 'BOB "AGAIN" CARNEY JR', 'BETSY HODGES', 'OLE SAVIOR', 'JAMES "JIMMY" L. STROUD, JR.', 'TONY LANE', 'EDMUND BERNARD BRUYERE', 'CHRISTOPHER CLARK', 'BILL KAHN', 'KURTIS W. HANNA', 'MIKE GOULD', 'CAM WINTON', 'MARK ANDREW', 'JOHN LESLIE HARTWIG', 'RAHN V. WORKCUFF', 'DON SAMUELS', 'STEPHANIE WOODRUFF', 'JOHN CHARLES WILSON', 'JOSHUA REA', 'JAMES EVERETT', 'CYD GORMAN', 'ALICIA K. BENNETT', 'TROY BENJEGERDES', 'CHRISTOPHER ROBIN ZIMMERMAN', 'JAYMIE KELLY', 'JEFFREY ALAN WAGNER', 'BOB FINE', 'MARK V ANDERSON', 'ABDUL M RAHAMAN "THE ROCK"', 'CAPTAIN JACK SPARROW', 'DAN COHEN', 'GREGG A. IVERSON', 'DOUG MANN', 'NEAL BAXTER', 'MERRILL ANDERSON')
Candidates who received votes: ('ABDUL M RAHAMAN "THE ROCK"', 'DAN COHEN', 'JAMES EVERETT', 'MARK V ANDERSON', 'TROY BENJEGERDES', 'ALICIA K. BENNETT', 'BETSY HODGES', 'MARK ANDREW', 'MIKE GOULD', 'BILL K

We know that Betsy Hodges won the 2013 election, so let's first check and make sure that 
we get the same result when we run the election using VoteKit:

In [20]:
minn_election = STV(profile=minneapolis_profile, m=1)
print(minn_election)

                                  Status  Round
BETSY HODGES                     Elected     35
MARK ANDREW                   Eliminated     34
DON SAMUELS                   Eliminated     33
CAM WINTON                    Eliminated     32
JACKIE CHERRYHOMES            Eliminated     31
BOB FINE                      Eliminated     30
DAN COHEN                     Eliminated     29
STEPHANIE WOODRUFF            Eliminated     28
MARK V ANDERSON               Eliminated     27
DOUG MANN                     Eliminated     26
OLE SAVIOR                    Eliminated     25
JAMES EVERETT                 Eliminated     24
ALICIA K. BENNETT             Eliminated     23
ABDUL M RAHAMAN "THE ROCK"    Eliminated     22
CAPTAIN JACK SPARROW          Eliminated     21
CHRISTOPHER CLARK             Eliminated     20
TONY LANE                     Eliminated     19
JAYMIE KELLY                  Eliminated     18
MIKE GOULD                    Eliminated     17
KURTIS W. HANNA               Eliminated

/mnt/efs/h/Dropbox/MADLAB/Training_Materials/June_25/.venv/lib/python3.13/site-packages/votekit/pref_profile/pref_profile.py:997: UserWarning: Profile does not contain rankings but max_ranking_length=3. Setting max_ranking_length to 0.
  warnings.warn(


The above output is generated after the final step of the election, so the output doesn't include any
information about how the election progressed.
But, if we were to step through each round of the election, we would see that the set of winners would
grow from the top down, the set of eliminated candidates would grow from the bottom up, and any
candidate that was neither elected nor eliminated would be listed in the middle as`Remaining`.

Let's walk through a smaller election, and see this in action.

In [4]:
candidates = ["A", "B", "C", "D", "E", "F", "G"]

ballots = [
    Ballot(ranking=[{"A"}, {"B"}], weight=3),
    Ballot(ranking=[{"B"}, {"C"}, {"D"}], weight=8),
    Ballot(ranking=[{"C"}, {"A"}, {"B"}], weight=1),
    Ballot(ranking=[{"D"}, {"E"}], weight=3),
    Ballot(ranking=[{"E"}, {"D"}, {"F"}], weight=1),
    Ballot(ranking=[{"F"}, {"G"}], weight=4),
    Ballot(ranking=[{"G"}, {"E"}, {"F"}], weight=3),
]

profile = PreferenceProfile(ballots=ballots)

print(profile)
print("Sum of ballot weights:", profile.total_ballot_wt)
print("Number of candidates:", len(profile.candidates))

election = STV(profile=profile, m=3)

print("Threshold:", election.threshold)
print("Number of rounds", len(election))
print(election)

Profile contains rankings: True
Maximum ranking length: 3
Profile contains scores: False
Candidates: ('A', 'B', 'C', 'D', 'E', 'F', 'G')
Candidates who received votes: ('A', 'B', 'C', 'D', 'E', 'F', 'G')
Total number of Ballot objects: 7
Total weight of Ballot objects: 23

Sum of ballot weights: 23
Number of candidates: 7
Initial tiebreak was unsuccessful, performing random tiebreak
Threshold: 6
Number of rounds 6
       Status  Round
B     Elected      1
D     Elected      4
F     Elected      6
A   Remaining      6
G  Eliminated      5
C  Eliminated      3
E  Eliminated      2


The `STV` method in the above code block returns an `Election` object that lets us access the
round-by-round information on what happens in the election.
The message about a tiebreak indicates that in some round, a random tiebreak was needed.

We can review it step-by-step instead of all at once. Just from a brief glance at the profile and threshold, we see that candidate B should be elected in the first round. Let's see this happen in two ways.

First, observe the first-place votes for each candidate. These are stored in the round 0 `ElectionState` object, which can be accessed in the following way:

In [5]:
election.election_states[0].scores

{'A': Fraction(3, 1),
 'B': Fraction(8, 1),
 'C': Fraction(1, 1),
 'D': Fraction(3, 1),
 'E': Fraction(1, 1),
 'F': Fraction(4, 1),
 'G': Fraction(3, 1)}

We can see from this that only B is over the threshold.  The other way we can see who wins in the first round is by looking at the next `ElectionState`.

In [6]:
print("elected", election.election_states[1].elected)
print()
print("eliminated", election.election_states[1].eliminated)
print()
print("remaining", election.election_states[1].remaining)

elected (frozenset({'B'}),)

eliminated (frozenset(),)

remaining (frozenset({'F'}), frozenset({'G', 'D', 'A', 'C'}), frozenset({'E'}))


$B$ passed the threshold by 2 votes with a total of 8, so the $B,C,D$ ballot is going to have $B$ removed and be given weight $2/8$ (excess/total) times its previous weight of 8. To check this, election objects have a method called `get_profile()` that returns the `PreferenceProfile` after a particular round.

In [7]:
election.get_profile(1)

Profile contains rankings: True
Maximum ranking length: 3
Profile contains scores: False
Candidates: ('G', 'A', 'C', 'F', 'D', 'E')
Candidates who received votes: ('C', 'D', 'E', 'A', 'F', 'G')
Total number of Ballot objects: 7
Total weight of Ballot objects: 17

Look, $B$ is now removed from all ballots, and the $B,C,D$ ballot became $C,D$ with weight 2. No one has enough votes to cross the 6 threshold, so the candidate with the least support will be eliminated---that is candidate $E$, with only one first-place vote.

The other way of accessing the round information for the election is using the `get_step()` method
as detailed below:

In [8]:
print("fpv after round 1:", election.election_states[1].scores)
print("go to the next step\n")

profile, state = election.get_step(2)
print("elected", state.elected)
print("\neliminated", state.eliminated)
print("\nremaining", state.remaining)
print(profile)

fpv after round 1: {'C': Fraction(3, 1), 'D': Fraction(3, 1), 'E': Fraction(1, 1), 'A': Fraction(3, 1), 'F': Fraction(4, 1), 'G': Fraction(3, 1)}
go to the next step

elected (frozenset(),)

eliminated (frozenset({'E'}),)

remaining (frozenset({'F', 'D'}), frozenset({'G', 'A', 'C'}))
Profile has been cleaned
Profile contains rankings: True
Maximum ranking length: 3
Profile contains scores: False
Candidates: ('F', 'G', 'A', 'D', 'C')
Candidates who received votes: ('C', 'D', 'A', 'F', 'G')
Total number of Ballot objects: 7
Total weight of Ballot objects: 17



In [9]:
print("fpv after round 2:", election.election_states[2].scores)
print("go to the next step\n")


print("elected", election.election_states[3].elected)
print("\neliminated", election.election_states[3].eliminated)
print("\nremaining", election.election_states[3].remaining)
print("\ntiebreak resolution", election.election_states[3].tiebreaks)
print()
print(election.get_profile(3))

fpv after round 2: {'C': Fraction(3, 1), 'D': Fraction(4, 1), 'A': Fraction(3, 1), 'F': Fraction(4, 1), 'G': Fraction(3, 1)}
go to the next step

elected (frozenset(),)

eliminated (frozenset({'C'}),)

remaining (frozenset({'D'}), frozenset({'F', 'A'}), frozenset({'G'}))

tiebreak resolution {frozenset({'G', 'A', 'C'}): (frozenset({'G'}), frozenset({'A'}), frozenset({'C'}))}

Initial tiebreak was unsuccessful, performing random tiebreak
Profile has been cleaned
Profile contains rankings: True
Maximum ranking length: 3
Profile contains scores: False
Candidates: ('F', 'G', 'A', 'D')
Candidates who received votes: ('D', 'A', 'F', 'G')
Total number of Ballot objects: 7
Total weight of Ballot objects: 17



Note that here, several candidates were tied for the fewest first-place votes at this stage. When this happens in STV, you use the first-place votes from the original profile to break ties. This means C will be eliminated.

### Modifying the Transfer Method of STV

There are several choices that can be made in the progression of a STV election. The most impactful
is the method used to transfer overvotes from one round to the next. The default method used here
is the "fractional transfer" method in which all of the ballots are reweighted according to the
number of surplus votes in the previous round and then the votes are transferred with that weight
to the next remaining candidate listed on the ballot.


In the "random transfer" method, we instead randomly select a subset of the ballots that voted
for the newly elected candidate and then transfer the votes from those ballots to the next place.

In [10]:
candidates = ["A", "B", "C", "D", "E", "F", "G"]

ballots = [
    Ballot(ranking=[{"A"}, {"B"}], weight=3),
    Ballot(ranking=[{"B"}, {"C"}, {"D"}], weight=8),
    Ballot(ranking=[{"B"}, {"D"}, {"C"}], weight=8),
    Ballot(ranking=[{"C"}, {"A"}, {"B"}], weight=1),
    Ballot(ranking=[{"D"}, {"E"}], weight=1),
    Ballot(ranking=[{"E"}, {"D"}, {"F"}], weight=1),
    Ballot(ranking=[{"F"}, {"G"}], weight=4),
    Ballot(ranking=[{"G"}, {"E"}, {"F"}], weight=1),
]

profile = PreferenceProfile(ballots=ballots)

print(profile)
print("Sum of ballot weights:", profile.total_ballot_wt)
print("Number of candidates:", len(profile.candidates))

election = STV(profile=profile, transfer=random_transfer, m=2)

print(election)

Profile contains rankings: True
Maximum ranking length: 3
Profile contains scores: False
Candidates: ('A', 'B', 'C', 'D', 'E', 'F', 'G')
Candidates who received votes: ('A', 'B', 'C', 'D', 'E', 'F', 'G')
Total number of Ballot objects: 8
Total weight of Ballot objects: 27

Sum of ballot weights: 27
Number of candidates: 7
Initial tiebreak was unsuccessful, performing random tiebreak
       Status  Round
B     Elected      1
D     Elected      7
F  Eliminated      6
C  Eliminated      5
A  Eliminated      4
G  Eliminated      3
E  Eliminated      2


## Elections

### Borda

In a Borda election, ranked ballots are converted to a score for a candidate, and then the candidates with the highest scores win. The traditional score vector is $(n,n-1,\dots,1)$: that is, if there are $n$ candidates, the first-place candidate on a ballot is given $n$ points, the second place $n-1$, all the way down to last, who is given $1$ point. You can change the score vector using the `score_vector` parameter.

In [11]:

candidates = ["A", "B", "C", "D", "E", "F"]

# recall IAC generates an "all bets are off" profile
iac = bg.ImpartialAnonymousCulture(candidates=candidates)
profile = iac.generate_profile(number_of_ballots=1000)

election = Borda(profile, m=3)

In [12]:

print(election.get_profile(0))
print()

print(election)

Profile contains rankings: True
Maximum ranking length: 6
Profile contains scores: False
Candidates: ('A', 'B', 'C', 'D', 'E', 'F')
Candidates who received votes: ('F', 'A', 'B', 'D', 'C', 'E')
Total number of Ballot objects: 421
Total weight of Ballot objects: 1000


      Status  Round
F    Elected      1
A    Elected      1
D    Elected      1
E  Remaining      1
C  Remaining      1
B  Remaining      1


In [13]:

# the winners up to the given round, -1 means final round
print("Winners:", election.get_elected(-1))

# the eliminated candidates up to the given round
print("Eliminated:", election.get_eliminated(-1))

# the ranking of the candidates up to the given round
print("Ranking:", election.get_ranking(-1))

# the outcome of the given round
print("Outcome of round 1:\n", election.get_status_df(1))

Winners: (frozenset({'F'}), frozenset({'A'}), frozenset({'D'}))
Eliminated: ()
Ranking: (frozenset({'F'}), frozenset({'A'}), frozenset({'D'}), frozenset({'E'}), frozenset({'C'}), frozenset({'B'}))
Outcome of round 1:
       Status  Round
F    Elected      1
A    Elected      1
D    Elected      1
E  Remaining      1
C  Remaining      1
B  Remaining      1


### **Try it yourself**

Using the following preference profile, try changing the score vector of a Borda election. Try replacing 3,2,1 with other Borda weights (decreasing and non-negative) showing that each candidate can be elected.

In [ ]:
ballots = [
    Ballot(ranking=[{"A"}, {"B"}, {"C"}], weight=3),
    Ballot(ranking=[{"A"}, {"C"}, {"B"}], weight=2),
    Ballot(ranking=[{"B"}, {"C"}, {"A"}], weight=2),
    Ballot(ranking=[{"C"}, {"B"}, {"A"}], weight=4),
]

profile = PreferenceProfile(ballots=ballots, candidates=["A", "B", "C"])

score_vector = # edit this!

election = Borda(profile, m=1, score_vector=score_vector)
print(election)

## Creating custom election systems

`VoteKit` can't be comprehensive in terms of possible election rules. However, with the `Election` and `ElectionState` classes, you can create your own. Let's create a bit of a silly example; to elect $m$ seats, at each stage of the election we randomly choose one candidate to elect. Most of the methods are handled by the `RankingElection` class, so we really only need to define how a step works, and how to know when it's over.


Here is an example where we elect a list of candidates in alphabetical order.

In [18]:
class AlphabeticaElection(RankingElection):
    """
    Simulates an election where we choose winners alphabetically at each stage.

    Args:
        profile (PreferenceProfile): Profile to run election on.
        m (int, optional): Number of seats to elect.
    """

    def __init__(self, profile: PreferenceProfile, m: int = 1):
        # the super method says call the RankingElection class
        self.m = m
        super().__init__(profile)

    def _is_finished(self) -> bool:
        """
        Determines if another round is needed.

        Returns:
            bool: True if number of seats has been met, False otherwise.
        """
        # need to unpack list of sets
        elected = [c for s in self.get_elected() for c in s]

        if len(elected) == self.m:
            return True

        return False

    def _run_step(
        self, profile: PreferenceProfile, prev_state: ElectionState, store_states=False
    ) -> PreferenceProfile:
        """
        Run one step of an election from the given profile and previous state.

        Args:
            profile (PreferenceProfile): Profile of ballots.
            prev_state (ElectionState): The previous ElectionState.
            store_states (bool, optional): True if `self.election_states` should be updated with the
                ElectionState generated by this round. This should only be True when used by
                `self._run_election()`. Defaults to False.

        Returns:
            PreferenceProfile: The profile of ballots after the round is completed.
        """

        candidates = sorted(profile.candidates)
        elected_cand = candidates[0]  # elect the first candidate alphabetically
        new_profile = remove_cand(elected_cand, profile)

        if store_states:
            self.election_states.append(
                ElectionState(
                    round_number = prev_state.round_number + 1,
                    remaining = (frozenset(new_profile.candidates),),
                    elected = (frozenset({elected_cand}),),
                )
            )
        return new_profile

In [19]:
cands = ["eggs", "toast", "apple", "blueberry", "oats", "coffee"]

profile = bg.ImpartialAnonymousCulture(candidates=cands).generate_profile(number_of_ballots=1000)

election = AlphabeticaElection(profile, m=3)

election

              Status  Round
apple        Elected      1
blueberry    Elected      2
coffee       Elected      3
oats       Remaining      3
toast      Remaining      3
eggs       Remaining      3

### Try it yourself!

Our goal in this is to try and recreate the STV with the random transfer method and make it into
a class called `RTV` (short for random transferrable vote).  We will be doing this in two stages:

1. Make a multi-round election that randomly selects a winner from each round.
2. Make the `RTV` class that randomly transfers votes from the elected candidate's supporting ballots
   to the next candidate's supporting ballots.

In [ ]:
class RandomWinners(RankingElection):
    """
    Simulates an election where we randomly choose winners at each stage.

    Args:
        profile (PreferenceProfile): Profile to run election on.
        m (int, optional): Number of seats to elect.
    """

    def __init__(self, profile: PreferenceProfile, m: int = 1):

        # Your code here

        raise NotImplementedError

    def _is_finished(self) -> bool:
        """
        Determines if another round is needed.

        Returns:
            bool: True if number of seats has been met, False otherwise.
        """

        # Your code here

        raise NotImplementedError

    def _run_step(
        self, profile: PreferenceProfile, prev_state: ElectionState, store_states=False
    ) -> PreferenceProfile:
        """
        Run one step of an election from the given profile and previous state.

        Args:
            profile (PreferenceProfile): Profile of ballots.
            prev_state (ElectionState): The previous ElectionState.
            store_states (bool, optional): True if `self.election_states` should be updated with the
                ElectionState generated by this round. This should only be True when used by
                `self._run_election()`. Defaults to False.

        Returns:
            PreferenceProfile: The profile of ballots after the round is completed.
        """

        # Your code here

        raise NotImplementedError

In [ ]:
candidates = ["A", "B", "C", "D", "E", "F"]
profile = bg.ImpartialCulture(candidates=candidates).generate_profile(1000)

election = RandomWinners(profile=profile, m=3)

In [ ]:
from typing import Callable, Union


# Assume no ties in the ballots when implementing this election.
# Assume that we will not need to handle simultaneous election issues.
# Remember all losing first-place votes are transferred to the next choice.
class RTV(RankingElection):
    """
    An election similar to STV, but where a random sample of `k` ballots are
    transferred to their next choice where `k` is the margin of victory for the
    candidate being elected (the number of votes passed the threshold).

    Args:
        profile (PreferenceProfile): Profile to run election on.
        m (int, optional): Number of seats to elect.
    """

    def __init__(self, profile: PreferenceProfile, m: int = 1):

        # Your code here
        
        raise NotImplementedError

    def _is_finished(self) -> bool:
        """
        Determines if another round is needed.

        Returns:
            bool: True if number of seats has been met, False otherwise.
        """

        # Your code here

        raise NotImplementedError

    def get_threshold(self, total_ballot_wt: Fraction) -> int:
        """
        Calculates threshold required for election.

        Args:
            total_ballot_wt (Fraction): Total weight of ballots to compute threshold.
        Returns:
            int: Value of the threshold.
        """
        return int(total_ballot_wt / (self.m + 1) + 1)  # Droop quota


    def _run_step(
        self, profile: PreferenceProfile, prev_state: ElectionState, store_states=False
    ) -> PreferenceProfile:
        """
        Run one step of an election from the given profile and previous state.

        Args:
            profile (PreferenceProfile): Profile of ballots.
            prev_state (ElectionState): The previous ElectionState.
            store_states (bool, optional): True if `self.election_states` should be updated with the
                ElectionState generated by this round. This should only be True when used by
                `self._run_election()`. Defaults to False.

        Returns:
            PreferenceProfile: The profile of ballots after the round is completed.
        """

        # Your code here

        raise NotImplementedError

Here are some profiles that you can test out the RTV method on.

In [ ]:
test_profile = PreferenceProfile(
    ballots=[
        Ballot(ranking=({"Orange"}, {"Pear"}), weight=3),
        Ballot(ranking=({"Pear"}, {"Strawberry"}, {"Cake"}), weight=8),
        Ballot(ranking=({"Strawberry"}, {"Orange"}, {"Pear"}), weight=1),
        Ballot(ranking=({"Cake"}, {"Chocolate"}), weight=3),
        Ballot(ranking=({"Chocolate"}, {"Cake"}, {"Burger"}), weight=1),
        Ballot(ranking=({"Burger"}, {"Chicken"}), weight=4),
        Ballot(ranking=({"Chicken"}, {"Chocolate"}, {"Burger"}), weight=3),
    ],
    max_ranking_length=3,
)

election = RTV(profile=test_profile, m=3)

election

In [ ]:
candidates = ["Orange", "Pear", "Strawberry", "Cake", "Chocolate", "Burger", "Chicken"]

profile = bg.ImpartialCulture(candidates=candidates).generate_profile(1000)


In [ ]:
# Run this a bunch to see if you get different results!
election = RTV(profile=profile, m=3)

election